Dear all,


I designed a simple exercise in order to speed up the basic learning you need to do before starting your master thesis. 


I want you to make a two layer network. You can use torch.nn.Linear, Conv1D or Conv2D. Your input and labels are randomly generated. So your input can be an image like WxHxC (width x height x channel), e.g. 4x5x3 or signal like WxC (width x channel) e.g. 5x4 and output is labels such as [0, 1, 2]. 

- Write the whole training and evaluation pipeline: You should calculate loss, backpropagate to calculate gradients and update your weights

- Save training and validation loss in tensorboard and monitor the graphs

- Save training and validation accuracy in tensorboard and monitor the graphs

- Visualize your activations

- Run it on your computer 

- Run it on Colab

- Run it on Snellius

- Push your code in your Github page together with Colab notebook and give the link to me before our meeting

- Bonus: Check if you can connect to Snellius or Google Colab via vscode. vscode has a very good debugger and editor. In this case, you can speed up your development. Here is some info for colab, but I have not tested myself. You don't have to use vscode. Whatever IDE you prefer is fine.


We can have a meeting in two weeks and see what kind of trouble or issues you encountered. Please reach out to each other for help. You can learn from my colab notebooks, pytorch tutorials such as this one, or chatGPT. 

Here is how to access Snellius GPU and how to connect.

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch import save, load
from torch.utils.data import DataLoader, random_split
from torch.optim import Adam, SGD
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt

Constructing training data, using an already existing dataset named MNIST    

In [18]:
#train = datasets.MNIST(root="data", download=True, train=True, transform=ToTensor())
#dataset = DataLoader(train, 32) #batches of 32 images

train = datasets.MNIST(root="data", download=True, train=True, transform=transforms.ToTensor())

# Split the dataset into training, test, and validation sets
total_size = len(train)
train_size = int(0.8 * total_size)  # 80% for training
test_size = (total_size - train_size) // 2  # 10% for testing
val_size = total_size - train_size - test_size  # 10% for validation

train_set, test_set, val_set = random_split(train, [train_size, test_size, val_size])

# Create DataLoader instances for each set
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False)


#shape of image is given as [1, 28, 28]
#classes are 0-9

commit_test = True

Create image classifier class

In [19]:
class ImageClassifier(nn.Module):
    def __init__(self):
        super().__init__() #ensures initialization logic of the nn.Module
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, (3,3)), #input channel = 1, since black and white. 32 filters. shape 3 by 3
            nn.ReLU(),
            nn.Conv2d(32, 64, (3,3)), #input channel is now 32, output = 64, still same shape
            nn.ReLU(),
            nn.Conv2d(64,64,(3,3)), #same
            nn.Flatten(), #always applied after last convolutional layer, converts output of convolutional layers into 1d tensor which can be used as input for a linear layer
            nn.Linear(64*(28-6)*(28-6), 10)# for each Conv2d we are deleting the outer two pixels layers of the data, since that how the pooling works. So we need to make up for that here by substracting 2 from each layer (2*3 = 6) from the original size (28)
        )

    def forward(self, x):
        return self.model(x)
    

Create an instance of the neural network, loss and optimizer. Also create platform to keep track of performance

In [20]:
clf = ImageClassifier().to('cpu') #use cuda if you have a separate GPU 
#opt = Adam(clf.parameters(), lr=1e-3) #Adam is one of the optimzers. Can use others. learningrate is now 0.001
opt = SGD(clf.parameters(), lr= 1e-3)
loss_fn = nn.CrossEntropyLoss() #possible loss function
#loss_fn = nn.MSELoss()
#RMSE_loss = torch.sqrt(loss_fn(prediction, target))
#RMSE_loss.backward()
writer = SummaryWriter('logs')  # 'logs' is the directory where TensorBoard logs will be stored
writer1 = SummaryWriter(f'runs/MNIST/tryingout_tensorboard')

Train flow

In [21]:
if __name__ == "__main__": #run deep learning for epochs
    loss_tracker = []
    for epoch in range(10): #train for 10 epochs
        for batch in train_loader:
            X, y = batch
            X, y = X.to('cpu'), y.to('cpu')
            yhat = clf(X) #this will be our predictions
            loss = loss_fn(yhat, y)

            #apply backpropagation
            opt.zero_grad()
            loss.backward() #calculate the gradients
            opt.step()
            writer.add_scalar('Loss/Train', loss, global_step=epoch)

        #print the loss for every batch
        loss_tracker.append(loss.item())
        plt.plot(loss_tracker)     
        plt.title('Loss over epochs')
        plt.xlabel('epoch')
        plt.ylabel('loss')
        print(f"Epoch: {epoch} loss is {loss.item()}")

        
        
    
    

Epoch: 0 loss is 0.22443144023418427
Epoch: 1 loss is 0.3028142750263214
Epoch: 2 loss is 0.05996200442314148
Epoch: 3 loss is 0.2948084771633148
Epoch: 4 loss is 0.3032251000404358
Epoch: 5 loss is 0.1654440462589264
Epoch: 6 loss is 0.15880417823791504
Epoch: 7 loss is 0.16517670452594757
Epoch: 8 loss is 0.391103059053421
Epoch: 9 loss is 0.15803726017475128


In [22]:
for batch in test_loader:
    X, y = batch
    X, y = X.to('cpu'), y.to('cpu')
    yhat = clf(X) #this will be our predictions
    loss = loss_fn(yhat, y)

    #apply backpropagation
    #opt.zero_grad()
    #loss.backward() #calculate the gradients
    #opt.step()
    writer.add_scalar('Loss/test', loss, global_step=epoch)
print(loss.item())

0.09312333911657333


In [23]:
tensorboard --logdir=logs


SyntaxError: cannot assign to expression here. Maybe you meant '==' instead of '='? (1983852640.py, line 1)

In [ ]:
#save
with open('model_state.pt', 'wb') as f: #wb = write binairy
    save(clf.state_dict(), f)